In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import fitz  
from textblob import TextBlob
from newspaper import Article
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Function to summarize text using LSA summarizer
def summarize_text(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 5)  # Summarize to 5 sentences
    return " ".join([str(sentence) for sentence in summary])

# Function to summarize content from a URL
def summarize_url():
    url = utext.get('1.0', "end").strip()  # Get URL from the text widget
    if not url:
        messagebox.showerror("Error", "URL cannot be empty")
        return

    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Enable text widgets to update content
        title.config(state="normal")
        author.config(state="normal")
        publication.config(state="normal")
        summary.config(state="normal")
        sentiment.config(state="normal")
        
        # Insert article details into the respective text widgets
        title.delete('1.0', 'end')
        title.insert('1.0', article.title)
        
        author.delete('1.0', 'end')
        author.insert('1.0', ', '.join(article.authors))
        
        publication.delete('1.0', 'end')
        publication.insert('1.0', article.publish_date)
        
        # Summarize the article text and display it
        summary_text = summarize_text(article.text)
        summary.delete('1.0', 'end')
        summary.insert('1.0', summary_text)
        
        # Perform sentiment analysis on the article text
        analysis = TextBlob(article.text)
        sentiment.delete('1.0', 'end')
        sentiment.insert('1.0', f'Polarity: {analysis.polarity}, Sentiment: {"positive" if analysis.polarity > 0 else "negative" if analysis.polarity < 0 else "neutral"}')
        
        # Disable text widgets after updating content
        title.config(state="disabled")
        author.config(state="disabled")
        publication.config(state="disabled")
        summary.config(state="disabled")
        sentiment.config(state="disabled")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Function to select a PDF file
def select_pdf():
    file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    if not file_path:
        return

    try:
        # Extract text from the selected PDF
        with fitz.open(file_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        analyze_pdf_text(text)
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Function to analyze text extracted from a PDF
def analyze_pdf_text(text):
    try:
        summary_text = summarize_text(text)
        analysis = TextBlob(text)
        sentiment_text = f'Polarity: {analysis.polarity}, Sentiment: {"positive" if analysis.polarity > 0 else "negative" if analysis.polarity < 0 else "neutral"}'

        # Display the summary and sentiment analysis on the Tkinter interface
        summary.config(state="normal")
        sentiment.config(state="normal")

        summary.delete('1.0', 'end')
        summary.insert('1.0', summary_text)
        sentiment.delete('1.0', 'end')
        sentiment.insert('1.0', sentiment_text)
        
        summary.config(state="disabled")
        sentiment.config(state="disabled")
        
        save_pdf(summary_text=summary_text, sentiment_text=sentiment_text)
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Function to save the summary and sentiment analysis to a PDF file
def save_pdf(summary_text, sentiment_text):
    save_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])
    if not save_path:
        return

    try:
        c = canvas.Canvas(save_path, pagesize=letter)
        width, height = letter
        margin = 40
        text_height = height - margin
        
        c.setFont("Helvetica", 12)
        c.drawString(margin, text_height, "Summary:")
        text_height -= 20
        
        # Properly wrap lines
        lines = summary_text.split('\n')
        for line in lines:
            while line:
                line_width = c.stringWidth(line, "Helvetica", 12)
                if line_width <= width - 2 * margin:
                    c.drawString(margin, text_height, line)
                    text_height -= 15
                    break
                else:
                    split_point = len(line)
                    while c.stringWidth(line[:split_point], "Helvetica", 12) > width - 2 * margin:
                        split_point -= 1
                    c.drawString(margin, text_height, line[:split_point])
                    text_height -= 15
                    line = line[split_point:]
                    if text_height < margin:
                        c.showPage()
                        c.setFont("Helvetica", 12)
                        text_height = height - margin
        
        if text_height < margin:
            c.showPage()
            c.setFont("Helvetica", 12)
            text_height = height - margin
            
        c.drawString(margin, text_height, "Sentiment Analysis:")
        text_height -= 20
        sentiment_lines = sentiment_text.split('\n')
        for line in sentiment_lines:
            while line:
                line_width = c.stringWidth(line, "Helvetica", 12)
                if line_width <= width - 2 * margin:
                    c.drawString(margin, text_height, line)
                    text_height -= 15
                    break
                else:
                    split_point = len(line)
                    while c.stringWidth(line[:split_point], "Helvetica", 12) > width - 2 * margin:
                        split_point -= 1
                    c.drawString(margin, text_height, line[:split_point])
                    text_height -= 15
                    line = line[split_point:]
                    if text_height < margin:
                        c.showPage()
                        c.setFont("Helvetica", 12)
                        text_height = height - margin
        
        c.save()
        messagebox.showinfo("Success", "PDF saved successfully")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Set up the Tkinter window
root = tk.Tk()
root.title("News Summarizer")
root.geometry('1200x700')

# Create and pack widgets for title
tlabel = tk.Label(root, text="Title")
tlabel.pack()
title = tk.Text(root, height=1, width=140)
title.config(state='disabled', bg='#dddddd')
title.pack()

# Create and pack widgets for author
alabel = tk.Label(root, text="Author")
alabel.pack()
author = tk.Text(root, height=1, width=140)
author.config(state='disabled', bg='#dddddd')
author.pack()

# Create and pack widgets for publication date
plabel = tk.Label(root, text="Publishing Date")
plabel.pack()
publication = tk.Text(root, height=1, width=140)
publication.config(state='disabled', bg='#dddddd')
publication.pack()

# Create and pack widgets for summary
slabel = tk.Label(root, text="Summary")
slabel.pack()
summary = tk.Text(root, height=20, width=140)
summary.config(state='disabled', bg="#dddddd")
summary.pack()

# Create and pack widgets for sentiment analysis
selabel = tk.Label(root, text="Sentiment Analysis")
selabel.pack()
sentiment = tk.Text(root, height=1, width=140)
sentiment.config(state='disabled', bg="#dddddd")
sentiment.pack()

# Create and pack widgets for URL input
ulabel = tk.Label(root, text="URL")
ulabel.pack()
utext = tk.Text(root, height=1, width=140)
utext.pack()

# Create and pack button for summarizing URL content
btn = tk.Button(root, text="Summarize URL", command=summarize_url)
btn.pack()

# Create and pack button for selecting PDF file
pdf_btn = tk.Button(root, text="Select PDF", command=select_pdf)
pdf_btn.pack()

# Run the Tkinter main loop
root.mainloop()
